In [52]:
import os
import time
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pytorch_lightning as pl

from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
'''ADD MORE IF NEEDED'''
from torch.nn import Module, Linear, ReLU, Conv2d, Sequential , CrossEntropyLoss
from  torch.utils.data import DataLoader, random_split
from torch.optim import Adam,lr_scheduler
from torchvision.transforms import Compose, ToTensor, Normalize 
''''''


''

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
'''
    ADD THE PATH TO VIEW IN TENSORBOARD
'''
PATH = ""
writer = SummaryWriter('runs/' + PATH)

In [8]:
batch_size = 64

In [50]:


class ClassModel(pl.LightningModule):
    def __init__(self):
        '''
            CHANGE TO APPLY (here 28*28 use for example MNIST)
        '''
        super(ClassModel, self).__init__()
        self.L1 = Linear(in_features=28*28, out_features=256) 
        self.L2 = Linear(in_features=256, out_features=128)
        self.L3 = Linear(in_features=128, out_features=64)
        self.L4 = Linear(in_features=64, out_features=10)
        self.relu = ReLU()

        '''
            NOT CHANGE HERE
        '''
        # self.train_dataset = None 
        # self.val_dataset = None 
        # self.test_dataset = None
        # self.optimizer= None 
        # self.step_lr_scheduler = None
        # self.losses = None
        # self.checkpoint = None
        self.outputs_train = []
        self.outputs_val = []
        self.outputs_test = []
    def forward(self, x):
        '''
        CHANGE TO APPLY
        '''
        x = self.L1(x)
        x = self.relu(x)
        x = self.L2(x)
        x = self.relu(x)
        x = self.L3(x)
        x = self.relu(x)
        x = self.L4(x)
        return x
    
    def prepare_data(self,percentage_val = 0.2):
        '''
            CHANGE TO APPLY
        '''
        name_dataset = torchvision.datasets.MNIST
        train_dataset_full = name_dataset(
            train=True,
            transform=Compose([ToTensor()]),
            root='./data/',
            download=True,
        )
        self.test_dataset = name_dataset(
            train=False,
            transform=Compose([ToTensor()]),
            root='./data/',
            download=False,
        )
        
        train_size = int((1.0-percentage_val)*len(train_dataset_full))
        val_size = int(percentage_val*len(train_dataset_full))
        self.train_dataset, self.val_dataset = random_split(train_dataset_full, lengths=[train_size,val_size])      
        return self.train_dataset, self.val_dataset, self.test_dataset
    
    def train_dataloader(self):
        '''
            CHANGE BATCH_SIZE TO APPLY
        '''
        train_dataset = DataLoader(
            dataset=self.train_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0
        )
        return train_dataset
        
    def val_dataloader(self):
        '''
            CHANGE BATCH_SIZE TO APPLY
        '''
        val_dataset = DataLoader(
            dataset=self.val_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=0
        )
        return val_dataset

    def test_dataloader(self):
        '''
            CHANGE BATCH_SIZE TO APPLY
        '''
        test_dataset = DataLoader(
            dataset=self.test_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=0
        )
        return test_dataset
    
    def configure_optimizers(self):
        '''
            CHANGE LEARNING_RATE, LOSS_ENTROPY,CLASS LR_SCHEDULER(step_size == epoch) TO APPLY
        '''
        learning_rate = 0.001
        optimizer = Adam(self.parameters(), lr=learning_rate)
        #self.losses = CrossEntropyLoss()
        #step_lr_scheduler = lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)
        return optimizer#{"optimizers": optimizer, "lr_scheduler": step_lr_scheduler}#, self.losses
    
    def training_step(self, batch, batch_idx):
        data, target = batch
        data = data.reshape(-1,28*28).to(device)
        target = target.to(device)
        output = self(data)
        loss = F.cross_entropy(output, target)
        _, predictions = torch.max(output, 1)
        accuracy = torch.sum(predictions == target).item()
        self.outputs_train.append(predictions)
        return {"loss":loss, "correct":accuracy}
    
    def validation_step(self, batch, batch_idx):
        data, target = batch
        data = data.reshape(-1, 28*28).to(device)
        target = target.to(device)
        output = self(data)
        loss = F.cross_entropy(output, target)
        _, predictions = torch.max(output, 1)
        accuracy = torch.sum(predictions == target).item()
        self.outputs_val.append(predictions)
        return {"val_loss":loss, "correct":accuracy}
    
    def test_step(self, batch, batch_idx):
        data, target = batch
        data = data.reshape(-1, 28*28).to(device)
        target = target.to(device)
        output = self(data)
        loss = F.cross_entropy(output, target)
        _, predictions = torch.max(output, 1)
        accuracy = torch.sum(predictions == target).item()
        self.outputs_test.append(predictions)
        return {"test_loss":loss, "correct":accuracy}
        
    # def on_train_epoch_end(self):
    #     # avg_loss = torch.stack([x['loss'] for x in self.outputs_train]).mean()
    #     # avg_correct = torch.stack([x['correct'] for x in self.outputs_train]).mean()
    #     all_correct = torch.stack(self.outputs_train)
    #     self.outputs_train.clear()
    #     return all_correct#{"avg_train_loss": avg_loss, "avg_correct": avg_correct}
    
    # def on_validation_epoch_end(self):
    #     # avg_loss = torch.stack([x['val_loss'] for x in self.outputs_val]).mean()
    #     # avg_correct = torch.stack([x['correct'] for x in self.outputs_val]).mean()
    #     all_correct = torch.stack(self.outputs_val)
    #     self.outputs_val.clear()
    #     return all_correct#{"avg_val_loss": avg_loss, "avg_correct": avg_correct}
    
    # def on_test_epoch_end(self):
    #     # avg_loss = torch.stack([x['test_loss'] for x in self.outputs_test]).mean()
    #     # avg_correct = torch.stack([x['correct'] for x in self.outputs_test]).mean()
    #     all_correct = torch.stack(self.outputs_test)
    #     self.outputs_test.clear()
    #     return all_correct#{"avg_test_loss": avg_loss, "avg_correct": avg_correct}

    # def save_model(self, PATH):
    #     return torch.save(self.state_dict(), PATH)
    
    # def load_state_dict(self, PATH):
    #     return self.load_state_dict(torch.load(PATH))

    # def take_checkpoint(self, epoch, loss, PATH):
    #     self.checkpoint ={
    #         'epoch': epoch,
    #         'model_state_dict': self.state_dict(),
    #         'optimizer_state_dict': self.optimizer.state_dict(),
    #         'loss': loss
    #     }
    #     return torch.save(self.checkpoint, PATH)
    
    # def load_checkpoint(self, PATH):
    #     self.checkpoint = torch.load(PATH)
    #     self.load_state_dict(self.checkpoint['model_state_dict'])
    #     self.optimizer.load_state_dict(self.checkpoint['optimizer_state_dict'])
    #     epoch = self.checkpoint['epoch']
    #     loss = self.checkpoint['loss']
    #     return epoch, loss

In [51]:
model = ClassModel()
trainer = pl.Trainer(max_epochs=2, fast_dev_run=False, accelerator="cpu")
trainer.fit(model)
trainer.test()

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name | Type   | Params | Mode 
----------------------------------------
0 | L1   | Linear | 200 K  | train
1 | L2   | Linear | 32.9 K | train
2 | L3   | Linear | 8.3 K  | train
3 | L4   | Linear | 650    | train
4 | relu | ReLU   | 0      | train
----------------------------------------
242 K     Trainable params
0         Non-trainable params
242 K     Total params
0.971     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
Restoring states from the checkpoint path at /Users/chessman/Desktop/ML_DL/Code_Tutorials/Numpy_Tensorflow_Pytorch_Tutorials/lightning_logs/version_1/checkpoints/epoch=1-step=1500.ckpt
Loaded model weights from the checkpoint at /Users/chessman/Desktop/ML_DL/Code_Tutorials/Numpy_Tensorflow_Pytorch_Tutorials/lightning_logs/version_1/checkpoints/epoch=1-step=1500.ckpt
/Users/chessman/miniforge3/envs/PyJupyter/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

[{}]